# Backpropagation via Matrix Calculus

We will provide written matrix calculus derivations and the implementation for a reverse-mode autodiff engine, then use it to implement and verify backpropagation across common layers and losses:
- Use matrix calculus (differentials, trace trick, chain rules) to derive gradients for:
  - affine $\rightarrow$ activation $\rightarrow$ softmax-CE
  - MSE regression
  - L2 regularization
- Implement a reverse-mode autodiff core (vector-Jacobian products) to reuse across layers
- Verify correctness with a single finite-difference check per op
- Train a simple 2-layer MLP on toy data (e.g. 2-moons) with SGD to demonstrate gradients flow

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=3)

## Matrix Calculus Basics

### Intro
- $\Delta$: big, finite step (difference)
- $d$: tiny, infinitesimal step (differential)
- $\partial$: derivative with respect to one variable (partial)
- $\nabla$: gradient operator (vector of partials)

### Differentials

$d(\cdot)$ means a differential, i.e. the infinitesimal change in a function when its inputs change a little. For a scalar function $f : \mathbb{R}^n \rightarrow \mathbb{R}$,

$$df(x) = \nabla f(x)^T dx \approx f(x + \Delta x) - f(x)$$

where:
- $dx$ is the vector of infinitesimal input changes
- $\nabla f(x)$ is the gradient (rate of change of $f$ with respect to each coordinate)

So $d f(x)$ is the infinitesimal change in the output, predicted linearly from the gradient and input change.

Let's consider the scalar function $f(x, y) = x^T Ay$. Let's see how $f$ changes when $x \mapsto x + dx, y \mapsto y + dy$.

$$
\begin{align}
d f(x, y) &= d(x^T Ay) \\
&= (x + dx)^T A (y + dy) - x^T Ay \\
&= dx^T Ay + x^T Ady + dx^T Ady \\
&= (Ay)^T dx + (A^T x)^T dy
\end{align}
$$

We drop $dx^T Ady$ because it is the product of two infinitesimals.

So the syntax $d(x^T Ay) = (Ay)^T dx + (A^T x)^T dy$ is a compact way of saying:
- the gradient wrt $x$ is $Ay$
- the gradient wrt $y$ is $A^T x$

### Trace trick

The trace of a square matrix is the sum of the diagonal elements.

$$\text{tr}(M) = \sum_i M_{ii}$$

The trace has a cyclic property meaning $\text{tr}(ABC) = \text{tr}(BCA) = \text{tr}(CAB)$ and linearity meaning $\text{tr}(A + B) = \text{tr}(A) + \text{tr}(B)$.

Gradients are sometimes easiest to express in terms of traces. The differential of a scalar function $f(X)$ with matrix input is often written:

$$
\begin{align}
df &= \sum_{i, j} \frac{\partial f}{\partial X_{ij}} dX_{ij} \\
&= \text{tr}((\frac{\partial f}{\partial X})^T dX) \\
&= \text{tr}(G^T dX)
\end{align}
$$

where $G$ is the gradient $\frac{\partial f}{\partial X}$.

For example, if we take the scalar function $f(X) = tr(A^T X)$, then its differential is $df = \text{tr}(A^T dX)$ and the gradient matrix with respect to $X$ is $A$.

### Example: Mini 2x2 Trace Trick

Let 

$$
X =
\begin{bmatrix}
x_{11} & x_{12} \\
x_{21} & x_{22}
\end{bmatrix}, \qquad
A =
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}, \qquad
f(X) = \text{tr}(A^T X)
$$

If we expand $f(X)$, we see that

$$
\begin{aligned}
f(X) &= 
\text{tr}\left(
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
x_{11} & x_{12} \\
x_{21} & x_{22}
\end{bmatrix}
\right) \\
&= 1 \cdot x_{11} + 2 \cdot x_{12} + 3 \cdot x_{21} + 4 \cdot x_{22}
\end{aligned}
$$

We can compute the gradient by inspection

$$
\nabla_X f =
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix} = A
$$

### Chain rules

Suppose we have a scalar loss $L$ that depends on an intermediate variable $Z$,
which itself depends on $Y$, which in turn depends on $X$:

$$
X \xrightarrow{g} Y \xrightarrow{f} Z \xrightarrow{} L
$$

We can always write the differential of the loss with respect to $Z$ as

$$
dL = \text{tr}(G_Z^T dZ)
$$

where $G_Z = \frac{\partial L}{\partial Z}$ is the upstream gradient arriving at $Z$.

If $Z = f(Y)$ and $Y = g(X)$, then by the multivariate chain rule:

$$
G_X = J_{Y \to X}^T J_{Z \to Y}^T G_Z = J_{Z \to X}^T G_Z
$$

where $J_{Z \to X} = \frac{\partial Z}{\partial X}$ is the Jacobian of $Z$ with respect to $X$.
This form emphasizes that the Jacobian is transposed when propagating gradients backwards.

In practice, the full Jacobian $J$ is rarely fully formed (which can be huge).
Instead, we compute the **vector-Jacobian product (VJP)** using structured rules (e.g. elementwise multiply for ReLU, matrix multiply for linear layers, broadcasting for sums/means):

$$
G_Y = J^T G_Z
$$

where:
- $J$ is the Jacobian of the local function $Z = f(Y)$
- $G_Z$ is the upstream gradient $\frac{\partial L}{\partial Z}$
- $G_Y$ is the downstream gradient $\frac{\partial L}{\partial Y}$, the result of the VJP

This is exactly what reverse-mode autodiff (backpropagation) implements:
it efficiently pushes $G$ backwards layer by layer without explicitly building the Jacobian.

- Forward mode: propagate differentials $dZ = J \, dX$
- Reverse mode: propagate gradients $G_X = J^T \, G_Z$
- Backprop = chaining many VJPs over a neural network

### Example: Mini 2x2 VJP

Let have a simple mapping $Z = f(Y)$ where both $Y, Z \in \mathbb{R}^2$.

$$
f\left(\begin{bmatrix} y_1 \\ y_2 \end{bmatrix}\right) = \begin{bmatrix} y_1 + 2y_2 \\ 3y_1 - y_2 \end{bmatrix}
$$

The Jacobian is

$$
J = 
\begin{bmatrix}
\frac{\partial z_1}{\partial y_1} & \frac{\partial z_1}{\partial y_2} \\[8pt]
\frac{\partial z_2}{\partial y_1} & \frac{\partial z_2}{\partial y_2}
\end{bmatrix} =
\begin{bmatrix}
1 & 2 \\
3 & -1
\end{bmatrix}
$$

Suppose the upstream gradient is

$$
G_Z = \begin{bmatrix}1 \\ -2\end{bmatrix}
$$

The VJP and $G_Y$, the downstream gradient wrt $Y$, is

$$
G_Y = J^T G_Z = 
\begin{bmatrix}
1 & 2 \\
3 & -1
\end{bmatrix}
\begin{bmatrix}
1 \\
-2
\end{bmatrix} =
\begin{bmatrix}
-5 \\
4
\end{bmatrix}
$$

### Broadcasting rules

When a smaller tensor is broadcasted to match a larger shape, the forward op duplicates values. In backprop, the gradient must be summed over the broadcasted dimensions to undo the duplication.

**Example (add bias):**
$$
Y = X + b, \quad X \in \mathbb{R}^{m \times n}, \quad b \in \mathbb{R}^n
$$

By definition

$$
dY_{ij} = dX_{ij} + db_j
$$

For a scalar loss $L(Y)$

$$
\begin{aligned}
dL &= \sum_{i,j} \frac{\partial L}{\partial Y_{ij}} dY_{ij} \\
&= \sum_{i,j} \frac{\partial L}{\partial Y_{ij}} (dX_{ij} + db_j) \\
&= \sum_{i,j} \frac{\partial L}{\partial Y_{ij}} dX_{ij} + \sum_{i,j} \frac{\partial L}{\partial Y_{ij}} db_j \\
&= \sum_{i,j} \frac{\partial L}{\partial Y_{ij}} dX_{ij} + \sum_{j} \left(\sum_{i} \frac{\partial L}{\partial Y_{ij}}\right) db_j
\end{aligned}
$$

By definition of differentials for a scalar $L$ as a function of $(X, b)$

$$
dL = \sum_{i,j} \frac{\partial L}{\partial X_{ij}} dX_{ij} + \sum_j \frac{\partial L}{\partial b_j} db_j
$$

So from the first coefficients, we see that $\nabla_X L = \nabla_Y L$. From the second term, we see that

$$
\frac{\partial L}{\partial X_{ij}} = \frac{\partial L}{\partial Y_{ij}} \implies \nabla_X L = \nabla_Y L
$$

$$
\frac{\partial L}{\partial b_j} = \sum_i \frac{\partial L}{\partial Y_{ij}} \implies \nabla_b L = \mathbf{1}^T \nabla_Y L
$$

where $\mathbf{1} \in \mathbb{R}^m$.

Forward: $b$ is broadcast to shape $(m, n)$

$$
\tilde{B} =
\begin{bmatrix}
b_1 & b_2 & \cdots & b_n \\
b_1 & b_2 & \cdots & b_n \\
\vdots & \vdots & \ddots & \vdots \\
b_1 & b_2 & \cdots & b_n
\end{bmatrix}
\in \mathbb{R}^{m \times n}
$$

Backward:
- $\nabla_X L = \nabla_Y L$
- $(\nabla_b L)_j = \sum_{i=1}^m \frac{\partial L}{\partial Y_{ij}}$

So the gradient wrt $b$ is the column-wise sum.

### Example: Mini 3x2 Broadcasting

Let

$$
X =
\begin{bmatrix}
x_{11} & x_{12} \\
x_{21} & x_{22} \\
x_{31} & x_{32}
\end{bmatrix}, \quad
b = \begin{bmatrix} b_1 & b_2 \end{bmatrix}
$$

The forward op is $Y = X + b$ where $b$ is broadcast to shape $3 \times 2$:

$$
\tilde{B} =
\begin{bmatrix}
b_1 & b_2 \\
b_1 & b_2 \\
b_1 & b_2
\end{bmatrix},
\quad
Y = X + \tilde{B}
$$

So explicitly

$$
Y =
\begin{bmatrix}
x_{11} + b_1 & x_{12} + b_2 \\
x_{21} + b_1 & x_{22} + b_2 \\
x_{31} + b_1 & x_{32} + b_2
\end{bmatrix}
$$

Suppose the upstream gradient is

$$
\nabla_Y L =
\begin{bmatrix}
g_{11} & g_{12} \\
g_{21} & g_{22} \\
g_{31} & g_{32}
\end{bmatrix}
$$

The gradient wrt $X$ and $b$ respectively are

$$
\nabla_X L = \nabla_Y L
$$

$$
\nabla_b L =
\begin{bmatrix}
g_{11} + g_{21} + g_{31} \\[6pt]
g_{12} + g_{22} + g_{32}
\end{bmatrix}
$$

### Reduction rules

When an operation reduces a tensor along some axis (e.g. sum, mean), the forward op collapses values. In backprop, the gradient must be broadcast back to the original shape to undo the reduction.

**Example (sum reduction):**

$$
y = \sum_{i} x_i, \quad x \in \mathbb{R}^m, \quad y \in \mathbb{R}
$$

By definition

$$
dy = \sum_{i} dx_i
$$

For a scalar loss $L(y)$

$$
\begin{aligned}
dL &= \frac{\partial L}{\partial y} \, dy \\
&= \frac{\partial L}{\partial y} \left(\sum_{i} dx_i\right) \\
&= \sum_{i} \left(\frac{\partial L}{\partial y}\right) dx_i
\end{aligned}
$$

By definition of differentials for a scalar $L$ as a function of $x$

$$
dL = \sum_{i} \frac{\partial L}{\partial x_i} dx_i
$$

Comparing coefficients, we see

$$
\frac{\partial L}{\partial x_i} = \frac{\partial L}{\partial y}
$$

So the gradient is broadcast:

$$
\nabla_x L = \mathbf{1} \nabla_y L
$$

where $\mathbf{1} \in \mathbb{R}^m$ is a vector of ones.

### Example: Mini 3x2 Reduction

Let $y$ be the column-wise sum

$$
X =
\begin{bmatrix}
1 & -2 \\
0 & 3 \\
4 & 1
\end{bmatrix} \in \mathbb{R}^{3\times 2}, \qquad
y = 
\begin{bmatrix}
1+0+4 \\
-2+3+1
\end{bmatrix}
=
\begin{bmatrix}
5 \\ 2
\end{bmatrix}
$$

Suppose the upstream gradient is
$$
\nabla_y L =
\begin{bmatrix}
2 \\ -1
\end{bmatrix}
$$

The gradient wrt $X$ is
$$
\nabla_X L =
\begin{bmatrix}
2 & -1 \\
2 & -1 \\
2 & -1
\end{bmatrix}
$$

## Layer and Loss Gradients

### Quadratic forms

Let $X \in \mathbb{R}^{N \times d}$, $W \in \mathbb{R}^{d \times k}$, and $Y \in \mathbb{R}^{N \times k}$. Define the loss:

$$f(X, W) = \frac{1}{2} \|XW - Y\|_F^2 = \frac{1}{2}\text{tr}((XW - Y)^T (XW - Y))$$

Differential:
$$
\begin{aligned}
df &= \text{tr}((XW - Y)^T (dX \, W + X \, dW)) \\
&= \text{tr}(((XW - Y)W^T)^T dX) + \text{tr}((X^T(XW - Y))^T dW)
\end{aligned}
$$

Gradients:
$$
\nabla_W f = X^T(XW - Y), \qquad \nabla_X f = (XW - Y)W^T
$$

### Softmax + CE (row-wise)

For a row of logits (raw scores) $z \in \mathbb{R}^k$, define the softmax:
$$
s = \text{softmax}(z)
$$

$$
s_i = \frac{e^{z_i}}{\sum_j e^{z_j}}
$$

Given a target distribution $y \in \mathbb{R}^k$ (often one-hot) or label, the
cross-entropy loss is
$$
\ell(z, y) = -\sum_i y_i \log s_i = -y^T \log s
$$

We know that

$$
\log s_i = z_i - \log \sum_j e^{z_j}
$$

Differential:
$$
\ell(z, y) 
= -y^Tz + (\sum_i y_i) (\log \sum_j e^{z_j})
= -y^Tz + \log \sum_j e^{z_j}
$$

$$
d\ell = -y + \frac{\mathbf{1}}{\sum_j e^{z_j}} \cdot (e^{z_1}, \dotsb, e^{z_k})^T = -y + s
$$

Gradient:
$$
\nabla_z \ell = s - y
$$

### Stable form softmax + CE (log-sum-exp trick)

Directly computing $\log \sum_j e^{z_j}$ can overflow if some $z_j$ are large. To fix this, subtract the max logit $m = \max_j z_j$ before exponentiating:

$$
\log \sum_j e^{z_j} = m + \log \sum_j e^{z_j - m}
$$

This keeps all exponentials in $[0, 1]$ and is numerically stable.

### Elementwise activations

Let $\phi$ be a scalar activation function (like ReLU, sigmoid, tanh, etc.) that is applied to each element of matrix $X \in \mathbb{R}^{m \times n}$ independently.

$$[\phi(X)]_{ij} = \phi(X_{ij})$$

Suppose the loss $L$ depends on $\phi(X)$, and you already know the upstream gradient
$$
G = \nabla_{\phi(X)} L \in \mathbb{R}^{m \times n}
$$

Then by the chain rule (elementwise), the gradient of $L$ with respect to $X$ is:
$$
\nabla_X L = G \odot \phi'(X)
$$
where $\odot$ denotes elementwise multiplication.

Special case: if the loss is a sum over all entries of $\phi(X)$, i.e.
$$
L = \sum_{ij} \phi(X_{ij})
$$

then $G = \mathbf{1}$ and:
$$
\nabla_X L = \phi'(X)
$$

Common activations and their derivatives:
- ReLU:

$$\phi(x) = \max(0, x), \qquad \phi'(x) = \mathbf{1}[x > 0]$$

- LeakyReLU ($\alpha$):

$$\phi(x) = \begin{cases} x & x > 0 \\ \alpha x & x \le 0 \end{cases}, \qquad
\phi'(x) = \begin{cases} 1 & x > 0 \\ \alpha & x \le 0 \end{cases}$$

- Sigmoid $\sigma(x)$:

$$\phi(x) = \frac{1}{1 + e^{-x}}, \qquad \phi'(x) = \sigma(x)(1 - \sigma(x))$$

- $\tanh$:

$$\phi(x) = \frac{e^x - e^{-x}}{e^x + e^{-x}}, \qquad \phi'(x) = 1 - \tanh^2(x)$$

- Softplus:

$$\phi(x) = \log(1 + e^x), \qquad \phi'(x) = \sigma(x)$$

### Affine layer

An affine layer is a lienar transformation of the input with weights and a bias, a basic "fully-connected" layer tha maps inputs into a new feature space before applying nonlinearities.

$$
Z = XW + \mathbf{1} b^T
$$

where:
- inputs $X \in \mathbb{R}^{N \times d}$ (each row is a sample)
- weights $W \in \mathbb{R}^{d \times k}$
- bias $b \in \mathbb{R}^k$ (broadcast to each row, $1 \in \mathbb{R}^N$ is a column of 1s)
- outputs $Z \in \mathbb{R}^{N \times k}$

Let's assume that the upstream gradient is $G = \nabla_Z L \in \mathbb{R}^{N \times k}$

Differential:

$$
\begin{aligned}
dZ &= dX W + X dW + \mathbf{1} (db)^T \\
dL &= \text{tr}(G^T dZ) \\
&= \text{tr}(G^T dX W) + \text{tr}(G^T X dW) + \text{tr}(G^T \mathbf{1} (db)^T) \\
&= \text{tr}((GW^T)^T dX) + \text{tr}((X^T G)^T dW) + (G^T \mathbf{1})^T db
\end{aligned}
$$

Gradients:

$$
\nabla_X L = G W^T \in \mathbb{R}^{N \times d}
$$

$$
\qquad \nabla_W L = X^T G \in \mathbb{R}^{d \times k}
$$

$$
\nabla_b L = G^T \mathbf{1} = \sum_{n=1}^N G_n \in \mathbb{R}^{k}
$$

If you augment inputs with a column of ones $\tilde{X} = \begin{bmatrix} X & \mathbf{1} \end{bmatrix}$ and $\tilde{W} = \begin{bmatrix} W \\ b^T \end{bmatrix}$, then $Z = \tilde{X} \tilde{W}$ and $\nabla_{\tilde{W}} L = \tilde{X}^T G$ and the last row recovers $\nabla_b L$.

### L2 weight decay